# Bir Hayvan Uzman Sisteminin Uygulanması

[Yeni Başlayanlar için YZ Müfredatı](http://github.com/microsoft/ai-for-beginners)'ndan bir örnek.

Bu örnekte, bazı fiziksel özelliklere dayalı olarak bir hayvanı belirlemek için basit bir bilgiye dayalı sistemi uygulayacağız. Sistem aşağıdaki VE-VEYA ağacı ile temsil edilebilir (bu, tüm ağacın bir parçasıdır, kolayca daha fazla kural ekleyebiliriz):

![](../images/AND-OR-Tree.png)

## Geriye dönük çıkarsamalı kendi uzman sistemlerimizin kabuğu

Üretme kurallarına dayalı bilgi temsili için basit bir dil tanımlamaya çalışalım. Kuralları tanımlamada anahtar kelimeler olarak Python sınıflarını kullanacağız. Temel olarak 3 tür sınıf olacaktır:
* `Sor`, kullanıcıya sorulması gereken bir soruyu temsil eder. Olası cevaplar kümesini içerir.
* `If` represents a rule, and it is just a syntactic sugar to store the content of the rule
* `Eger` bir kuralı temsil eder ve sadece kuralın içeriğini saklamak için sözdizimsel bir bileşendir.
* `VE`/`VEYA`, ağacın VE/VEYA dallarını temsil eden sınıflardır. Sadece argümanların listesini içeride saklarlar. Kodu basitleştirmek için, tüm işlevler `Icerik` üst sınıfında tanımlanmıştır.

In [ ]:
class Sor():
    def __init__(self,tercihler=['e','h']):
        self.tercihler = tercihler
    def sor(self):
        if max([len(x) for x in self.tercihler])>1:
            for i,x in enumerate(self.tercihler):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.tercihler[x]
        else:
            print("/".join(self.tercihler),flush=True)
            return input()

class Icerik():
    def __init__(self,x):
        self.x=x
        
class Eger(Icerik):
    pass

class VE(Icerik):
    pass

class VEYA(Icerik):
    pass

Sistemimizde, çalışan bellek, **nitelik-değer çiftleri** olarak **olguların (gerçeklerin)** listesini içerir. Bilgi tabanı, eylemleri (çalışan belleğe eklenmesi gereken yeni gerçekleri) VE-VEYA ifadeleri olarak ifade edilen koşullara eşleyen büyük bir sözlük olarak tanımlanabilir. Ayrıca, bazı gerçekler `Sor`-ulabilir.

In [ ]:
kurallar = {
    'varsayilan': Sor(['e','h']),
    'renk' : Sor(['kirmizi-kahverengi','siyah ve beyaz','diger']),
    'desen' : Sor(['koyu seritler','koyu benekler']),
    'memeli': Eger(VEYA(['killi','sut verir'])),
    'etcil': Eger(VEYA([VE(['sivri disler','penceler','ileriye bakan gozler']),'et yer'])),
    'toynakli': Eger(['memeli',VEYA(['toynaklari var','gevis getirir'])]),
    'kus': Eger(VEYA(['tuylu',VE(['ucar','yumurta verir'])])),
    'hayvan:maymun' : Eger(['memeli','etcil','renk:kirmizi-kahverengi','desen:koyu benekler']),
    'hayvan:kaplan' : Eger(['memeli','etcil','renk:kirmizi-kahverengi','desen:koyu seritler']),
    'hayvan:zurafa' : Eger(['toynakli','uzun boyun','uzun bacaklar','desen:koyu benekler']),
    'hayvan:zebra' : Eger(['toynakli','desen:koyu seritler']),
    'hayvan:devekusu' : Eger(['kus','uzun boyun','renk:siyah ve beyaz','ucamaz']),
    'hayvan:penguen' : Eger(['kus','yuzer','renk:siyah ve beyaz','ucamaz']),
    'hayvan:albatros' : Eger(['kus','iyi ucar'])
}

Geriye dönük çıkarsama gerçekleştirmek için `BilgiTabani` sınıfını tanımlayacağız. Şunları içerecektir:
* Çalışan `bellek` - nitelikleri değerlerle eşleştiren bir sözlük
* Yukarıda tanımlanan biçimde `kurallar` bilgi tabanı

İki ana yöntem şunlardır:
* Bir niteliğin değerini elde etmek için `getir`, gerekirse çıkarım yapar. Örneğin, `getir('renk')` bir renk yuvasının değerini alır (gerekirse sorar ve değeri daha sonra kullanmak üzere çalışan bellekte saklar). `getir('renk:mavi')` diye sorarsak, bir renk isteyecek ve ardından renge bağlı olarak `e`/`h` değerini döndürecektir.
* `degerlendir` gerçek çıkarsamayı gerçekleştirir, yani VE/VEYA ağacında ilerler, alt hedefleri değerlendirir, vb.

In [ ]:
class BilgiTabani():
    def __init__(self,kurallar):
        self.kurallar = kurallar
        self.bellek = {}
        
    def getir(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.getir(k)
            return 'e' if v==vv else 'h'
        if name in self.bellek.keys():
            return self.bellek[name]
        for fld in self.kurallar.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'e' if fld==name else fld.split(':')[1]
                res = self.degerlendir(self.kurallar[fld],field=name)
                if res!='e' and res!='h' and value=='e':
                    self.bellek[name] = res
                    return res
                if res=='e':
                    self.bellek[name] = value
                    return value
        # field is not found, using default
        res = self.degerlendir(self.kurallar['varsayilan'],field=name)
        self.bellek[name]=res
        return res
                
    def degerlendir(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Sor):
            print(field)
            return expr.sor()
        elif isinstance(expr,Eger):
            return self.degerlendir(expr.x)
        elif isinstance(expr,VE) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,VE) else expr
            for x in expr:
                if self.degerlendir(x)=='h':
                    return 'h'
            return 'e'
        elif isinstance(expr,VEYA):
            for x in expr.x:
                if self.degerlendir(x)=='e':
                    return 'e'
            return 'h'
        elif isinstance(expr,str):
            return self.getir(expr)
        else:
            print("Bilinmeyen ifade: {}".format(expr))

Şimdi hayvan bilgi tabanımızı tanımlayalım ve incelemeyi gerçekleştirelim. Bu aramanın size sorular soracağını unutmayın. Evet-hayır soruları için `e`/`h` yazarak veya daha uzun çoktan seçmeli cevaplar için sayı (0..N) belirterek cevap verebilirsiniz.

In [ ]:
kb = BilgiTabani(kurallar)
kb.getir('hayvan')

## İleri Çıkarsama için PyKnow'u Kullanma

Sonraki örnekte, bilgi temsili kütüphanelerinden biri olan [PyKnow](https://github.com/buguroo/pyknow/) kullanarak ileriye çıkarsama uygulamaya çalışacağız. **PyKnow**, klasik eski 
[CLIPS](http://www.clipsrules.net/index.html) sistemine benzer şekilde tasarlanmış, Python'da ileriye çıkarsama sistemleri oluşturmaya yönelik bir kütüphanedir.

Kendimiz ileriye zincirlemeyi pek sorun yaşamadan da uygulayabilirdik, ancak saf uygulamalar genellikle çok verimli değildir. Daha etkili kural eşleştirmesi için özel bir algoritma olan [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) kullanılır.

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

In [ ]:
from pyknow import *

Sistemimizi `KnowledgeEngine` (BilgiMotoru) altsınıfları olan bir sınıf olarak tanımlayacağız. Her kural, kuralın ne zaman tetikleneceğini belirten `@Rule` (Kural) ek açıklamalı ayrı bir işlevle tanımlanır. Kuralın içinde, `declare` (bildirme) işlevini kullanarak yeni gerçekler ekleyebiliriz ve bu gerçekleri eklemek, ileri çıkarsama motoru tarafından daha fazla kuralın çağrılmasıyla sonuçlanacaktır.

In [ ]:
class Hayvanlar(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sivri disler'),Fact('penceler'),Fact('ileriye bakan gozler')),
           Fact('et yer')))
    def etcil(self):
        self.declare(Fact('etcil'))
        
    @Rule(OR(Fact('killi'),Fact('sut verir')))
    def memeli(self):
        self.declare(Fact('memeli'))

    @Rule(Fact('memeli'),
          OR(Fact('toynaklari var'),Fact('gevis getirir')))
    def toynaklar(self):
        self.declare('toynakli')
        
    @Rule(OR(Fact('tuylu'),AND(Fact('ucar'),Fact('yumurta verir'))))
    def kus(self):
        self.declare('kus')
        
    @Rule(Fact('memeli'),Fact('etcil'),
          Fact(renk='kirmizi-kahverengi'),
          Fact(desen='koyu benekler'))
    def maymun(self):
        self.declare(Fact(hayvan='maymun'))

    @Rule(Fact('memeli'),Fact('etcil'),
          Fact(renk='kirmizi-kahverengi'),
          Fact(desen='koyu seritler'))
    def kaplan(self):
        self.declare(Fact(hayvan='kaplan'))

    @Rule(Fact('toynakli'),
          Fact('uzun boyun'),
          Fact('uzun bacaklar'),
          Fact(desen='koyu benekler'))
    def zurafa(self):
        self.declare(Fact(hayvan='zurafa'))

    @Rule(Fact('toynakli'),
          Fact(desen='koyu seritler'))
    def zebra(self):
        self.declare(Fact(hayvan='zebra'))

    @Rule(Fact('kus'),
          Fact('uzun boyun'),
          Fact('ucamaz'),
          Fact(renk='siyah ve beyaz'))
    def devekusu(self):
        self.declare(Fact(hayvan='devekusu'))

    @Rule(Fact('kus'),
          Fact('yuzer'),
          Fact('ucamaz'),
          Fact(renk='siyah ve beyaz'))
    def penguen(self):
        self.declare(Fact(hayvan='penguen'))

    @Rule(Fact('kus'),
          Fact('iyi ucar'))
    def albatros(self):
        self.declare(Fact(hayvan='albatros'))
        
    @Rule(Fact(hayvan=MATCH.a))
    def print_result(self,a):
          print('Hayvan {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Bir bilgi tabanı tanımladıktan sonra, çalışan belleğimizi bazı ilk gerçeklerle doldururuz ve ardından çıkarsama gerçekleştirmek için `run()` yöntemini çağırırız. Sonuç olarak, hayvanla ilgili son gerçek de dahil olmak üzere (başlangıçtaki tüm gerçekleri doğru bir şekilde kurarsak) çalışan belleğe yeni çıkarsanan gerçeklerin eklendiğini görebilirsiniz.

In [ ]:
ex1 = Hayvanlar()
ex1.reset()
ex1.factz([
    Fact(renk='kirmizi-kahverengi'),
    Fact(desen='koyu seritler'),
    Fact('sivri disler'),
    Fact('penceler'),
    Fact('ileriye bakan gozler'),
    Fact('sut verir')])
ex1.run()
ex1.facts